# Práctico 7

In [181]:
import numpy as np
from scipy.stats import chi2, norm
from random import random
import math

## Ejercicio 1

- a)

In [152]:
probs = np.array([0.25, 0.5, 0.25])
frec_obs = np.array([141, 291, 132])
n = sum(frec_obs)
frec_esp = [p_i * n for p_i in probs]

# Estadistico T
T = 0
for i in range(3):
    T += (frec_obs[i] - frec_esp[i])**2 / frec_esp[i]
    

df = len(probs) - 1
p_valor = chi2.sf(T, df)

print(f"p_valor = {p_valor:.4f}")

p_valor = 0.6500


- b)

In [153]:
def multinomial(n, p):
    res = [0] * len(p)
    for _ in range(n):
        x = random()
        if x < p[0]:
            res[0] += 1
        elif x < p[0] + p[1]:
            res[1] += 1
        else:
            res[2] += 1
    return res

Nsim = 1000
p = 0

# simulacion del p-valor
for _ in range(Nsim):
    obs = multinomial(n, probs)
    t_sim = 0
    for i in range(3):
        t_sim += (obs[i] - frec_esp[i])**2 / frec_esp[i]
        
    if t_sim >= T:
        p += 1

print("p-valor simulado:", p / Nsim)

p-valor simulado: 0.654


## Ejercicio 2

In [154]:
# a)

probs = 1/6
frec_obs = [158, 172, 164, 181, 160, 165]
n = 1000
frec_esp = 1/6 * n

T = 0
for i in range(6):
    T += (frec_obs[i] - frec_esp) ** 2 / frec_esp
    
print(T)
gl = 6 - 1 # Grados de libertad
p_valor = chi2.sf(T, gl)

print(f"p_valor = {p_valor:.4f}")

2.18
p_valor = 0.8237


In [155]:
# b)

def binomial(n, p):
    i = 0
    for _ in range(n):
        if random() < p:
            i += 1
    return i

"""
x_1 = binomial(1000, 1/6)
x_2 = binomial(1000 - x_1, 1/5)
x_3 = binomial(1000 - (x_1 + x_2), 1/4)
x_4 = binomial(1000 - (x_1 + x_2 + x_3), 1/3)
x_5 = binomial(1000 - (x_1 + x_2 + x_3 + x_4), 1/2)
x_6 = 1000 - (x_1 + x_2 + x_3 + x_4 + x_5)

print("1:", x_1)
print("2:", x_2)
print("3:", x_3)
print("4:", x_4)
print("5:", x_5)
print("6:", x_6)
print(x_1 + x_2 + x_3 + x_4 + x_5 + x_6)"""

def sim(Nsim=1000):
    p_valor = 0
    for _ in range(Nsim):
        
        # Genero la frecuencia observada de la tirada de dados
        frec_muestral = []
        tasas_riesgo = [1/6, 1/5, 1/4, 1/3, 1/2]
        total_tiradas = 0
        for k in range(6-1):
            x_k = binomial(n - total_tiradas, tasas_riesgo[k])
            total_tiradas += x_k
            frec_muestral.append(x_k)
        frec_muestral.append(n - total_tiradas)
        
        t_sim = 0
        for s in range(6):
            t_sim += (frec_muestral[s] - frec_esp)**2 / frec_esp
        
        if t_sim >= T:
            p_valor += 1
            
    return p_valor / Nsim
            
print("p-valor simulado:", sim())


p-valor simulado: 0.831


## Ejercicio 3

In [156]:
# Kolmogorov-Smirnov con Uniformes
def K_S_unif(datos):
    n = len(datos)
    datos.sort()
    d = 0
    for j in range(n):
        x = datos[j]
        # D = max_{1<=j<=n} {j/n - F(X_j), F(X_j) - (j-1)/n}
        d = max(d, (j+1)/n - x, x-j/n)
    return d

In [157]:
datos_ej3 = [0.12, 0.18, 0.06, 0.33, 0.72, 0.83, 0.36, 0.27, 0.77, 0.74]
#datos_ej3_a = [0.12, 0.98] * 5
#print(datos_ej3_a)
n = len(datos_ej3)
d = K_S_unif(datos_ej3)

def sim(Nsim=10000):
    p_valor = 0
    for _ in range(Nsim):
        uniformes = np.random.uniform(0, 1, n)
        d_j = K_S_unif(uniformes)
        if d_j >= d:
            p_valor += 1
    return p_valor / Nsim

print("Estimador de Kolmogorv-Smirnov D =", d)
print("p-valor simulado=", sim())

Estimador de Kolmogorv-Smirnov D = 0.24


p-valor simulado= 0.5323


## Ejercicio 4


In [158]:
def F_exponencial(x, lambda_):
    return 1 - np.exp(-x*lambda_)

# Kolmogorov-Smirnov con exponenciales
def K_S_exponencial(datos, lambda_):
    n = len(datos)
    datos.sort()
    d = 0
    for j in range(n):
        x = datos[j]
        F_x = F_exponencial(x, lambda_)
        # D = max_{1<=j<=n} {j/n - F(X_j), F(X_j) - (j-1)/n}
        d = max(d, (j+1)/n - F_x, F_x-j/n)
    return d

In [159]:
datos_ej4 = [86, 133, 75, 22, 11, 144, 78, 122, 8, 146, 33, 41, 99]
n = len(datos_ej4)
l = 1/50
d = K_S_exponencial(datos_ej4, l)

def sim(Nsim=10000):
    p_valor = 0
    for _ in range(Nsim):
        # Como no estimamos parametros usamos uniformes
        uniformes = np.random.uniform(0, 1, n)
        d_j = K_S_unif(uniformes)
        if d_j >= d:
            p_valor += 1
    return p_valor / Nsim

print("Estimador de Kolmogorv-Smirnov D =", d)
p_valor = sim()
print("p-valor simulado=", p_valor)


print("si el nivel de rechazo es de 5%;")
if p_valor < 0.05:
    print("Hipotesis nula rechazada")
else:
    print("Hipotesis nula no rechazada")

Estimador de Kolmogorv-Smirnov D = 0.3922544552361856


p-valor simulado= 0.0252
si el nivel de rechazo es de 5%;
Hipotesis nula rechazada


## Ejercicio 5

In [160]:
datos = [6, 7, 3, 4, 7, 3, 7, 2, 6, 3, 7, 8, 2, 1, 3, 5, 8, 7]
n = len(datos)
k = 9 # Cantidad de intervalos o número de agrupamiento de los valores 

p = (sum(datos)/n) / 8 # Estimación de p para una Binomial(8, p)
print("Parametro p estimado:", p)

# Probabilidades teoricas p_i
probs = []
for i in range(9):
    comb = math.comb(8, i)
    probs.append(comb * p**i * (1-p)**(8-i))

# Frecuencias observadas N_i
frec_obs = [0]*k
for obs in datos:
    frec_obs[obs] += 1

# Estadístico de prueba
T = 0
for i in range(k):
    frec_esp = n * probs[i]
    T += (frec_obs[i] - frec_esp)**2 / frec_esp
    
print("T de Pearson:", T)
gl = k - 1 - 1 # Grados de libertad
p_valor = chi2.sf(T, 7)
print(f"p-valor: {p_valor:.5f}")

Parametro p estimado: 0.6180555555555556
T de Pearson: 31.499330934155324
p-valor: 0.00005


## Ejercicio 6

In [161]:
probs = [0.31, 0.22, 0.12, 0.10, 0.08, 0.06, 0.04, 0.04, 0.02, 0.01]
frec_obs = [188, 138, 87, 65, 48, 32, 30, 34, 13, 2]
n = sum(frec_obs)
k = 10 # Numero de términos

T = 0
for i in range(k):
    frec_esp = probs[i] * n
    T += (frec_obs[i] - frec_esp)**2 / frec_esp
print("T de Pearson:", T)
gl = k - 1 # Grados de libertad
p_valor = chi2.sf(T, gl)
print(f"p-valor: {p_valor:.4f}")

T de Pearson: 9.810370888711903
p-valor: 0.3661


**Haciendo una simulacion**

In [162]:
probs = [0.31, 0.22, 0.12, 0.10, 0.08, 0.06, 0.04, 0.04, 0.02, 0.01] # del 1 al 10
def premios():
    u = random()
    i = 0
    p = probs[i]
    while u > p:
        i += 1
        p += probs[i]
    return i+1

def sim(Nsim=10000):
    p_valor = 0
    
    for _ in range(Nsim):
        datos = []
        for _ in range(n):
            datos.append(premios())
            
        N = [0]*k # Frecuencias observadas en la simulacion
        for obs in datos:
            N[obs-1] += 1
            
        T_sim = 0
        for i in range(k):
            frec_esp = n * probs[i]
            T_sim += (N[i] - frec_esp)**2 / frec_esp
        
        if T_sim >= T:
            p_valor += 1
    
    return p_valor / Nsim

print("p-valor simulado:", sim())

p-valor simulado: 0.3637


## Ejercicio 7

In [173]:
from random import random

n = 30

# Muestra de 30 exponenciales con lambda=1
Y = [(-np.log(random())) for i in range(n)]
Y.sort()

# Calculo el estadistico de K - S porque los datos son continuos
d = K_S_exponencial(Y, 1) # lambda=1
print("Estimador de Kolmorogov-Smirnov D =", d)

# H0: La muestra proviene de una distribucion exponencial con media 1

# Calculo del p-valor, con Nsim = 1000
Nsim = 1000
p_valor = 0
for i in range(Nsim):
    # usando uniformes:
    Y = np.random.uniform(0, 1, n)
    d_i = K_S_unif(Y)
    
    # Usando exponenciales: solo si el lambda fuera estimado si o si tendriamos que usar esta forma:
    # Y = [(-np.log(random())) for i in range(n)]
    # d_i = K_S_exponencial(Y, n/sum(Y))
    # Tambien esta bien conociendo lambda, pero es menos eficiente
    
    if d_i >= d:
        p_valor += 1
estimacion = p_valor/Nsim

if estimacion > 0.05:
    print(f"No rechazar H_0, pues p-valor= {estimacion:.4f} > 0.05")
else:
    print(f"Rechazar H_0, pues p-valor= {estimacion:.4f} < 0.05")
    

Estimador de Kolmorogov-Smirnov D = 0.12267763943821963


No rechazar H_0, pues p-valor= 0.7290 > 0.05


## Ejercicio 8

In [164]:
def norm_F(x):
    if x >= 0:
        Fi = math.erf(x / np.sqrt(2.)) / 2.+0.5
    else:
        Fi = 0.5-math.erf(-x / np.sqrt(2.)) / 2.
    return Fi

def K_S_norm(datos):
    n = len(datos)
    datos.sort()
    d = 0
    for j in range(n):
        x = datos[j]
        d = max(d, (j+1)/n - norm_F(x), norm_F(x) - j/n)
    return d

import random
import math

def rt(df): # df grados de libertad
    x = random.gauss(0.0, 1.0)
    y = 2.0 * random.gammavariate(0.5*df, 2.0)
    return x / np.sqrt(y/df)


**(a)**

In [165]:
df = 11 # Grados de libertad

print()
print('n\t  ', 'D\t ', '\t p-valor')
n_muestra = [10, 20, 100, 1000]
for n in n_muestra:
    muestra = []
    for _ in range(n):
        muestra.append(rt(df))
    D = K_S_norm(muestra)
    
    Nsim = 10000
    p_valor = 0
    for _ in range(Nsim):
        uniformes = np.random.uniform(0, 1, n) # Usamos uniformes porque conocemos los parámetros
        uniformes.sort()
        d_j = 0
        for j in range(n):
            u_j = uniformes[j]
            d_j = max(d_j, (j+1)/n - u_j, u_j - j/n)
        if d_j >= D:
            p_valor += 1
    print(f'{n}\t {D:.5f}\t {(p_valor/Nsim):.5f}')


n	   D	  	 p-valor


10	 0.19639	 0.76660
20	 0.24092	 0.15930
100	 0.10846	 0.17130
1000	 0.09023	 0.00000


**(b)**

In [166]:
df = 11 # Grados de libertad

print()
print('n\t  ', 'D\t ', '\t p-valor', '\t Nsim')
n_muestra = [10, 20, 100]#, 1000]
for n in n_muestra:
    muestra = []
    for _ in range(n):
        muestra.append(rt(df))
    D = K_S_norm(muestra)
    
    L = 0.01 # Amplitud del intervalo
    z_alpha_2 = 1.96 # Si la confianza es del 95%
    d = L/(2*z_alpha_2)
    Nsim = 0
    p_valor = 0
    while Nsim <= 100 or np.sqrt(1/Nsim) > d:
        Nsim += 1
        uniformes = np.random.uniform(0, 1, n) # Usamos uniformes porque conocemos los parámetros
        uniformes.sort()
        d_j = 0
        for j in range(n):
            u_j = uniformes[j]
            d_j = max(d_j, (j+1)/n - u_j, u_j - j/n)
        if d_j >= D:
            p_valor += 1
    print(f'{n}\t {D:.5f}\t {(p_valor/Nsim):.5f}\t {Nsim}')
    # n	        D	  	 p-valor
    # 1000	 0.08756	 0.00001


n	   D	  	 p-valor 	 Nsim
10	 0.28034	 0.34421	 153664
20	 0.24328	 0.15923	 153664
100	 0.16438	 0.00795	 153664


## ejercicio 9

In [176]:
n = 15

# Muestra de 15 exponenciales con lambda desconocida
Y = [1.6, 10.3, 3.5, 13.5, 18.4, 7.7, 24.3, 10.7, 8.4, 4.9, 7.9, 12, 16.2, 6.8, 14.7]
Y.sort()

# Calculo el estadistico de K - S porque los datos son continuos
lambda_est = n / sum(Y)
d = K_S_exponencial(Y, lambda_est)
print("Estimador de Kolmorogov-Smirnov D =", d)

# H0: La muestra proviene de una distribucion exponencial con media desconocida

# Calculo del p-valor, con Nsim = 1000
_lambda = 1 / lambda_est
p_valor = 0
Nsim = 10000
for _ in range(Nsim):
    muestra = []
    # usando exponenciales:
    for _ in range(n):
        muestra.append(-np.log(1 - random()) / _lambda)
    muestra.sort()
    _lambda_est = n / sum(muestra)
    d_sim = K_S_exponencial(muestra, _lambda_est)
    
    if d_sim >= d:
        p_valor += 1
estimacion = p_valor/Nsim

if estimacion > 0.05:
    print(f"No rechazar H_0, pues p-valor= {estimacion:.4f} > 0.05")
else:
    print(f"Rechazar H_0, pues p-valor= {estimacion:.4f} < 0.05")
    

Estimador de Kolmorogov-Smirnov D = 0.26949936321059237
No rechazar H_0, pues p-valor= 0.0501 > 0.05


## Ejercicio 10

In [185]:
# Datos
muestra_obs = [91.9, 97.8, 111.4, 122.3, 105.4, 95.0, 103.8, 99.6, 96.6, 119.3, 104.8, 101.7]
n = len(muestra_obs)

# Estimacion de parametros
mu_est = np.mean(muestra_obs)
sigma_est = np.std(muestra_obs, ddof=1)

# estandarizamos los datos con los parámetros estimados
Z = [(x - mu_est) / sigma_est for x in muestra_obs]

# Estadístico observado
d = K_S_norm(Z)
print("D observado:", d)


p_valor = 0
Nsim = 10000
for _ in range(Nsim):
    # Generar muestra normal con mu y sigma estimados
    muestra = np.random.normal(loc=mu_est, scale=sigma_est, size=n)
    
    # Estandarizar muestra
    mu_sim = np.mean(muestra)
    sigma_sim = np.std(muestra, ddof=1)
    Z_sim = [(x - mu_sim) / sigma_sim for x in muestra]

    # Calcular estadístico simulado    
    d_sim = K_S_norm(Z_sim)
    
    if d_sim >= d:
        p_valor += 1
        
estimacion = p_valor/Nsim

if estimacion > 0.05:
    print(f"No rechazar H_0, pues p-valor= {estimacion:.4f} > 0.05")
else:
    print(f"Rechazar H_0, pues p-valor= {estimacion:.4f} < 0.05")


D observado: 0.19638944697995542
No rechazar H_0, pues p-valor= 0.2237 > 0.05
